In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

from sklearn import preprocessing

In [2]:
data = pd.read_csv('retail_train.csv')
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
...,...,...
2037,2496,[6534178]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813..."


In [ ]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])

    recs = np.random.choice(items, size=n, p=weights, replace=False)
    
    return recs.tolist()

In [13]:
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()

In [22]:
# x_array = np.array(items_weights['sales_value'])
# x_array
# normalized_arr = preprocessing.normalize([x_array])
# normalized_arr

In [24]:
items_weights['sales_value'] = np.log2(items_weights['sales_value'] + 0.001)
items_weights['sales_value'] = items_weights['sales_value'] - items_weights['sales_value'].min()
items_weights['sales_value'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights = items_weights.rename(columns={'sales_value': 'weight'})
items_weights

,item_id,weight
0,25671,0.000012
1,26081,0.000008
2,26093,0.000009
3,26190,0.000009
4,26355,0.000009
...,...,...
86860,17381856,0.000000
86861,17382205,0.000011
86862,17383227,0.000010
86863,17827644,0.000010


In [26]:
%%time

# your_code
result['w_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

CPU times: user 2.88 s, sys: 0 ns, total: 2.88 s
Wall time: 2.88 s


,user_id,actual,w_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[838948, 934030, 825829, 2768012, 41590]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[952965, 940454, 15982771, 998206, 56098]"


In [27]:
result.head()

,user_id,actual,w_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[838948, 934030, 825829, 2768012, 41590]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[952965, 940454, 15982771, 998206, 56098]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[848773, 9420325, 1017602, 957374, 8068463]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1132677, 990216, 1052391, 6551867, 5565415]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[879215, 816237, 889536, 10210756, 13157933]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [ ]:
# result = pd.read_csv('../predictions/predictions_basic.csv')
# result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [28]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [31]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [35]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [32]:
items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

,user_id,actual,w_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[838948, 934030, 825829, 2768012, 41590]","[6904878, 1887729, 8293582, 15971732, 975148]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[952965, 940454, 15982771, 998206, 56098]","[27933, 13775839, 962845, 9485585, 1724733]"


In [33]:
# your_code

rec_metrics = {}
for col in result.columns[1:]:
    rec_metrics[col] = round(result.apply(lambda row: precision_at_k(row[col], row['actual']), axis=1).mean(), 4)

In [34]:
pd.DataFrame(pd.Series(rec_metrics)).T

,actual,w_random_recommendation,random_recommendation
0,1.0,0.0008,0.0006


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code